# Phase 2
# Market Basket Insights with Advanced Association Rule Learning and Visualization

 # 1) Import Libraries

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

# 2) Data Pre-processing

In [ ]:
df_ = pd.read_csv("../input/market-basket-analysis/Assignment-1_Data.csv", sep = ";")
df = df_.copy()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head(10)

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
5,536365,SET 7 BABUSHKA NESTING BOXES,2,01.12.2010 08:26,"7,65",17850.0,United Kingdom
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6,01.12.2010 08:26,"4,25",17850.0,United Kingdom
7,536366,HAND WARMER UNION JACK,6,01.12.2010 08:28,"1,85",17850.0,United Kingdom
8,536366,HAND WARMER RED POLKA DOT,6,01.12.2010 08:28,"1,85",17850.0,United Kingdom
9,536367,ASSORTED COLOUR BIRD ORNAMENT,32,01.12.2010 08:34,"1,69",13047.0,United Kingdom


In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)

    print("##################### Types #####################")
    print(dataframe.dtypes)

    print("##################### Head #####################")
    print(dataframe.head(head))

    print("##################### Tail #####################")
    print(dataframe.tail(head))

    print("##################### NA #####################")
    print(dataframe.isnull().sum())

In [ ]:
check_df(df)

##################### Shape #####################
(522064, 7)
##################### Types #####################
BillNo         object
Itemname       object
Quantity        int64
Date           object
Price          object
CustomerID    float64
Country        object
dtype: object
##################### Head #####################
   BillNo                             Itemname  Quantity              Date  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  01.12.2010 08:26   
1  536365                  WHITE METAL LANTERN         6  01.12.2010 08:26   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  01.12.2010 08:26   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  01.12.2010 08:26   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  01.12.2010 08:26   

  Price  CustomerID         Country  
0  2,55     17850.0  United Kingdom  
1  3,39     17850.0  United Kingdom  
2  2,75     17850.0  United Kingdom  
3  3,39     17850.0  United Kingdom  
4  3,39     

In [ ]:
# Drop na values
df.dropna(inplace=True)
# Quantity and Price should be  greater than 0
df = df[df["Quantity"] > 0]
# We have to change the price column datatype as a numeric
df ['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df = df[df["Price"] > 0]

In [ ]:
check_df(df)

##################### Shape #####################
(1537, 7)
##################### Types #####################
BillNo         object
Itemname       object
Quantity        int64
Date           object
Price         float64
CustomerID    float64
Country        object
dtype: object
##################### Head #####################
      BillNo                            Itemname  Quantity              Date  \
45    536370                             POSTAGE         3  01.12.2010 08:45   
237   536392  RUSTIC  SEVENTEEN DRAWER SIDEBOARD         1  01.12.2010 10:29   
377   536403                             POSTAGE         1  01.12.2010 11:27   
1113  536527                             POSTAGE         1  01.12.2010 13:04   
4348  536779                        Bank Charges         1  02.12.2010 15:08   

      Price  CustomerID         Country  
45     18.0     12583.0          France  
237   165.0     13705.0  United Kingdom  
377    15.0     12791.0     Netherlands  
1113   18.0     12662.0 

# 3) Exploratory Data Analysis and Some Visualizations

In [ ]:
total_sales = df
total_sales["Total_Price"] = total_sales["Price"] * total_sales["Quantity"]
#total_sales.columns
total_sales_per_customer = total_sales.groupby(["CustomerID", "Country"]).agg({"Total_Price": "sum"})
total_sales_per_customer.head(10)

,,Total_Price
CustomerID,Country,
12349.0,Italy,300.0
12350.0,Norway,40.0
12352.0,Norway,280.0
12356.0,Portugal,324.0
12357.0,Switzerland,25.0
12358.0,Austria,240.0
12360.0,Austria,360.0
12361.0,Belgium,15.0
12362.0,Belgium,489.0


## Top 10 Shoppers and Their Coutries

In [ ]:
total_sales_per_customer.reset_index(inplace=True)
total_sales_per_customer.sort_values(by = "Total_Price", ascending = False).head(10)

,CustomerID,Country,Total_Price
577,17450.0,United Kingdom,10496.0
471,15581.0,United Kingdom,2750.0
67,12471.0,Germany,2400.0
412,14607.0,United Kingdom,2120.0
112,12540.0,Spain,1820.0
246,12748.0,United Kingdom,1788.0
460,15482.0,United Kingdom,1646.0
414,14646.0,Netherlands,1458.0
201,12681.0,France,1422.0
198,12678.0,France,1297.0


In [ ]:
# consider that for all time period
data_fig = total_sales_per_customer.sort_values(by = "Total_Price", ascending = False).head(100)

fig = px.scatter(data_fig, x="CustomerID", y="Total_Price",
	         size="Total_Price", color="Country",
                 hover_name="Country", log_x=True, size_max=60, title="Top 100 Shoppers and Their Contries & Shopping Amounts")
fig.show()

In [ ]:
# consider that for all time period
#total_sales_per_customer.head(20)
total_sales_per_customer.groupby(["Country"]).agg({"Total_Price":"sum"}).reset_index().sort_values(by="Total_Price", ascending=False )

,Country,Total_Price
17,United Kingdom,53159.0
4,Germany,21155.0
3,France,15713.0
14,Spain,5917.0
2,Belgium,4269.0
16,Switzerland,4027.0
10,Norway,2996.0
12,Portugal,2508.0
9,Netherlands,1947.0
7,Italy,1663.0


In [ ]:
# consider that for all time period
data = total_sales_per_customer.groupby(["Country"]).agg({"Total_Price":"sum"}).reset_index().sort_values(by="Total_Price", ascending=False )
fig = px.bar(data, x='Country', y='Total_Price' , title = "Shopping Amounts by Countries")
fig.show()

# 4) Invoice Product Matrix

In [ ]:
#df_united_kingdom = df.loc[df["Country"]=="United Kingdom"]
df_invoice_product_matrix = df.groupby(['BillNo', 'Itemname']).  \
    agg({"Quantity": "sum"}).unstack().fillna(0).  \
    applymap(lambda x: 1 if x > 0 else 0)

df_invoice_product_matrix.head(10)

Quantity                             \
Itemname BEADED CHANDELIER T-LIGHT HOLDER BILI NUT AND WOOD NECKLACE   
BillNo                                                                 
536370                                  0                          0   
536392                                  0                          0   
536403                                  0                          0   
536527                                  0                          0   
536779                                  0                          0   
536835                                  0                          0   
536840                                  0                          0   
536852                                  0                          0   
536858                                  0                          0   
536861                                  0                          0   

                                                            \
Itemname BLING KEY RING STAND BOTANICAL GARDENS WALL CLOCK   
BillNo                                                       
536370                      0                            0   
536392                      0                            0   
536403                      0                            0   
536527                      0                            0   
536779                      0                            0   
536835                      0                            0   
536840                      0                            0   
536852                      0                            0   
536858                      0                            0   
536861                      0                            0   

                                                               \
Itemname BREAD BIN DINER STYLE PINK BREAD BIN DINER STYLE RED   
BillNo                                                          
536370                            0                         0   
536392                            0                         0   
536403                            0                         0   
536527                            0                         0   
536779                            0                         0   
536835                            0                         0   
536840                            0                         0   
536852                            0                         0   
536858                            0                         0   
536861                            0                         0   

                                                                     \
Itemname BROWN CHECK CAT DOORSTOP BROWN KUKUI COCONUT SEED NECKLACE   
BillNo                                                                
536370                          0                                 0   
536392                          0                                 0   
536403                          0                                 0   
536527                          0                                 0   
536779                          0                                 0   
536835                          0                                 0   
536840                          0                                 0   
536852                          0                                 0   
536858                          0                                 0   
536861                          0                                 0   

                                                         ...  \
Itemname Bank Charges CAKE STAND VICTORIAN FILIGREE MED  ...   
BillNo                                                   ...   
536370              0                                 0  ...   
536392              0                                 0  ...   
536403              0                                 0  ...   
536527              0                                 0  ...   
536779              1                                 0  ...   
536835 

# 5) Assosiciation Rules Learning

In [ ]:
frequent_itemsets = apriori(df_invoice_product_matrix, min_support=0.001, use_colnames=True) 
frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
30,0.715076,"((Quantity, POSTAGE))"
27,0.054633,"((Quantity, Next Day Carriage))"
3,0.043568,"((Quantity, BOTANICAL GARDENS WALL CLOCK))"
26,0.031120,"((Quantity, Manual))"
23,0.027663,"((Quantity, LOVE SEAT ANTIQUE WHITE METAL))"
...,...,...
573,0.001383,"((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua..."
574,0.001383,"((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua..."
575,0.001383,"((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua..."
576,0.001383,"((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua..."


In [ ]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.001)
rules.sort_values("support", ascending=False).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
123,"((Quantity, VINTAGE BLUE KITCHEN CABINET))","((Quantity, VINTAGE RED KITCHEN CABINET))",0.012448,0.024896,0.006916,0.555556,22.314815,0.006606,2.193983
122,"((Quantity, VINTAGE RED KITCHEN CABINET))","((Quantity, VINTAGE BLUE KITCHEN CABINET))",0.024896,0.012448,0.006916,0.277778,22.314815,0.006606,1.367380
17,"((Quantity, SPOTTED WHITE NATURAL SEED NECKLACE))","((Quantity, BROWN KUKUI COCONUT SEED NECKLACE))",0.003458,0.004149,0.002766,0.800000,192.800000,0.002752,4.979253
14,"((Quantity, BROWN KUKUI COCONUT SEED NECKLACE))","((Quantity, RED KUKUI COCONUT SEED NECKLACE))",0.004149,0.007607,0.002766,0.666667,87.636364,0.002735,2.977178
137,"((Quantity, BROWN KUKUI COCONUT SEED NECKLACE)...","((Quantity, RED KUKUI COCONUT SEED NECKLACE))",0.002766,0.007607,0.002766,1.000000,131.454545,0.002745,inf
138,"((Quantity, RED KUKUI COCONUT SEED NECKLACE), ...","((Quantity, BROWN KUKUI COCONUT SEED NECKLACE))",0.002766,0.004149,0.002766,1.000000,241.000000,0.002755,inf
139,"((Quantity, BROWN KUKUI COCONUT SEED NECKLACE))","((Quantity, RED KUKUI COCONUT SEED NECKLACE), ...",0.004149,0.002766,0.002766,0.666667,241.000000,0.002755,2.991701
140,"((Quantity, RED KUKUI COCONUT SEED NECKLACE))","((Quantity, BROWN KUKUI COCONUT SEED NECKLACE)...",0.007607,0.002766,0.002766,0.363636,131.454545,0.002745,1.567082
141,"((Quantity, BILI NUT AND WOOD NECKLACE))","((Quantity, BROWN KUKUI COCONUT SEED NECKLACE)...",0.002766,0.002766,0.002766,1.000000,361.500000,0.002759,inf
119,"((Quantity, SPOTTED WHITE NATURAL SEED NECKLACE))","((Quantity, RED KUKUI COCONUT SEED NECKLACE))",0.003458,0.007607,0.002766,0.800000,105.163636,0.002740,4.961964


In [ ]:
sorted_rules = rules.sort_values("lift", ascending=False)
rules.sort_values("lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
113,"((Quantity, MARIE ANTOINETTE TRINKET BOX GOLD))","((Quantity, MARIE ANTOINETTE TRINKET BOX SILVER))",0.001383,0.001383,0.001383,1.000000,723.0,0.001381,inf
114,"((Quantity, VANILLA SCENT CANDLE JEWELLED BOX))","((Quantity, OCEAN SCENT CANDLE IN JEWELLED BOX))",0.001383,0.001383,0.001383,1.000000,723.0,0.001381,inf
112,"((Quantity, MARIE ANTOINETTE TRINKET BOX SILVER))","((Quantity, MARIE ANTOINETTE TRINKET BOX GOLD))",0.001383,0.001383,0.001383,1.000000,723.0,0.001381,inf
115,"((Quantity, OCEAN SCENT CANDLE IN JEWELLED BOX))","((Quantity, VANILLA SCENT CANDLE JEWELLED BOX))",0.001383,0.001383,0.001383,1.000000,723.0,0.001381,inf
38147,"((Quantity, DOORMAT UNION FLAG), (Quantity, DO...","((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua...",0.002075,0.001383,0.001383,0.666667,482.0,0.001380,2.995851
38078,"((Quantity, DOORMAT UNION FLAG), (Quantity, DO...","((Quantity, DOORMAT SPOTTY HOME SWEET HOME), (...",0.001383,0.002075,0.001383,1.000000,482.0,0.001380,inf
38067,"((Quantity, DOORMAT UNION FLAG), (Quantity, DO...","((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua...",0.002075,0.001383,0.001383,0.666667,482.0,0.001380,2.995851
38068,"((Quantity, DOORMAT UNION FLAG), (Quantity, DO...","((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua...",0.002075,0.001383,0.001383,0.666667,482.0,0.001380,2.995851
38069,"((Quantity, DOORMAT UNION FLAG), (Quantity, DO...","((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua...",0.002075,0.001383,0.001383,0.666667,482.0,0.001380,2.995851
38070,"((Quantity, DOORMAT MERRY CHRISTMAS RED), (Qua...","((Quantity, DOORMAT UNION FLAG), (Quantity, DO...",0.001383,0.002075,0.001383,1.000000,482.0,0.001380,inf


In [ ]:
# We can try below products in the loop
# ('Quantity', 'VANILLA SCENT CANDLE JEWELLED BOX')
# ('Quantity', 'DOORMAT MERRY CHRISTMAS RED')
# ('Quantity', 'DOORMAT RESPECTABLE HOUSE')
# ('Quantity', 'DOORMAT SPOTTY HOME SWEET HOME')
# ('Quantity', 'DOORMAT UNION FLAG')
# ('Quantity', 'DOORMAT UNION FLAG')

recommendation_list = []

for i, product in sorted_rules["antecedents"].items():
    for j in list(product):
        if j == ('Quantity', 'MARIE ANTOINETTE TRINKET BOX GOLD'):
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))

#### As a result, the products that are purchased together with the ***('Quantity', 'MARIE ANTOINETTE TRINKET BOX GOLD')*** product are seen below.

In [ ]:
recommendation_list

[[('Quantity', 'DOORMAT MERRY CHRISTMAS RED'),
  ('Quantity', 'DOORMAT NEW ENGLAND'),
  ('Quantity', 'DOORMAT HEARTS')]]